In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q transformers

In [3]:
import torch

class ScoredToken():
  """
  Represents a token and a corresponding score, which should roughly translate to the likelihood this
  token should be appended to some given generation sequence.
  """
  def __init__(self, token_id, score):
    self.token_id = token_id
    self.score = score

  def __str__(self):
    return f"{self.token_id}: {self.score: .8f}"
  
  def __repr__(self):
    return self.__str__()

class GeneratedSequence():
  """
  Represents a sequence in the process of being generated; an initial token, a potential end token, and a series of 
  ScoredTokens between them. This class also maintains the overall sequence score, which is the cumulative probability 
  of this generated sequence being the best output given some query.
  """
  def __init__(self, tokenizer, initial_token, end_token_id, initial_score):
    self.tokenizer = tokenizer
    self.end_token_id = end_token_id
    self._score = initial_score # Cumulative log probs of this sequence
    self.normalized_score = initial_score
    self.sequence = [ScoredToken(initial_token, initial_score)]
  
  def append(self, scored_token):
    """
    Append the given ScoredToken to this sequence; add its log-probability to this
    sequence's total cumulative log-prob
    """
    self.sequence.append(scored_token)
    self._score += scored_token.score
    self.normalized_score = self._score / len(self.sequence)

  def ids(self):
    return [st.token_id for st in self.sequence]

  def tokens(self):
    return self.tokenizer.decode(torch.tensor(self.ids()), skip_special_tokens=True)
  
  def has_ended(self):
    """
    Returns True if the last token in this sequence is the end-of-sequence token ID
    """
    return self.sequence and self.sequence[-1].token_id == self.end_token_id

  def __str__(self):
    return f"{self._score: .8f}({self.normalized_score: .8f}): {self.sequence}"

  def __repr__(self):
    return self.__str__()
  
  def __copy__(self):
    gs = GeneratedSequence(self.tokenizer, None, self.end_token_id, 0.0)
    gs.sequence = self.sequence.copy()
    gs._score = self._score
    gs.normalized_score = self.normalized_score
    return gs

  def __iter__(self):
    return self.sequence.__iter__()
  
  def __lt__(self, other_sequence):
   return self.normalized_score < other_sequence.normalized_score

  def __le__(self, other_sequence):
    return self.normalized_score <= other_sequence.normalized_score

  def __eq__(self, other_sequence):
    return self.normalized_score - other_sequence.normalized_score <= 1e-5 and self.ids() == other_sequence.ids()
  
  def __ne__(self, other_sequence):
    return self.normalized_score - other_sequence.normalized_score > 1e-5 or self.ids() != other_sequence.ids()
  
  def __gt__(self, other_sequence):
    return self.normalized_score > other_sequence.normalized_score
  
  def __ge__(self, other_sequence):
    return self.normalized_score >= other_sequence.normalized_score

In [4]:
import torch

def greedy_search(model, input_ids, max_length, start_token_id, end_token_id, device):

  # The output sequence starts as a tensor containing just the start token ID
  output_sequence = torch.full((1, 1), start_token_id, device=device)

  for _ in range(max_length - 1):
    # Predict next token, given input context IDs and output sequence so far
    output = model(input_ids=input_ids, decoder_input_ids=output_sequence)

    # Extract the raw model output
    logits = output.logits[:, -1, :]

    # Extract the argmax, the highest-scoring element, from the model output logits.
    # Unsqueeze to remove the singular batch dimension.
    next_token_id = logits.argmax(-1).unsqueeze(-1)

    # Concatenate the next token ID into the output sequence
    output_sequence = torch.cat([output_sequence, next_token_id], dim=-1)

    # If the next token happens to be the end-of-sequence token, break the loop
    if (next_token_id == end_token_id):
      break

  # Return greedily-computed output token sequence
  return output_sequence

In [5]:
from collections import Counter
from copy import deepcopy
import math
import torch
import tqdm

def beam_search(
    tokenizer,
    model,
    src_input_ids,
    start_token_id,
    end_token_id,
    device,
    beam_width=8,
    temperature=1.0,
    max_length=64,
    decay_repeated=False):
  """
  Performs a beam search to generate heuristically-determined best top-k text sequences from a model.

  Args:
  tokenizer (Tokenizer): Tokenizer instance used for token manipulation.
  model (Model): The model used for generating predictions.
  src_input_ids (torch.Tensor): Source input ids for which output needs to be generated.
  start_token_id (int): The token id used to start the sequence generation.
  end_token_id (int): The token id that indicates the end of a sequence.
  device (torch.device): The device (CPU/GPU) where tensors should be allocated.
  beam_width (int, optional): The number of sequences to keep at each step of the beam search. Default is 8.
  temperature (float, optional): A factor used to model the confidence of the predictions. 
                                   A higher temperature results in more diversified outputs. Default is 1.0.
  max_length (int, optional): The maximum length of the sequence to be generated. 
                                   Default is 64.
  decay_repeated (bool, optional): Flag to apply decay to the score of repeated tokens to discourage repetition. Default is False.

  Returns:
  list: A list of generated sequences, where each sequence is represented as an object containing sequence ids, score, 
    and tokenized representation.

  Description:
  This function initiates a beam search algorithm to generate sequences from a given model. The search begins with a sequence 
  containing only the start_token_id. At each step, the model predicts the next token for each sequence in the current set of 
  candidate sequences. The top 'beam_width' tokens and their probabilities are used to extend the current sequences. This process 
  repeats until the sequences reach the maximum specified length or all sequences end with the end_token_id. 

  Optionally, the score for repeated tokens can be decayed to discourage repetition. 
  Finally, the scores of the sequences can be normalized by their length to prevent penalizing longer sequences.
  """
  
  print(f"beam search | k = {beam_width} bos = {start_token_id} eos = {end_token_id} temp = {temperature} beam_width = {beam_width}")

  # The initial candidate sequence is simply the start token ID with a sequence score of 0
  candidate_sequences = [GeneratedSequence(tokenizer, start_token_id, end_token_id, 0.0)]

  # Build up output sequences until max_length tokens are reached
  for _ in tqdm.tqdm(range(max_length)):

    # Temporary list to store candidates for the next generation step
    next_step_candidates = []

    # Iterate through all candidate sequences; for each, generate the next most likely tokens
    # and add them to the next-step sequnce of candidates
    for candidate in candidate_sequences:
      if not candidate.has_ended(): # skip candidate sequences which have included the end-of-sequence token

        # Build a tensor out of the candidate IDs; add a single batch dimension
        gen_seq = torch.tensor(candidate.ids(), device=device).unsqueeze(0)

        # Predict next token
        output = model(input_ids=src_input_ids, decoder_input_ids=gen_seq)

        # Extract logits from output
        logits = output.logits[:, -1, :]

        # Scale logits using temperature value
        scaled_logits = logits / temperature
        
        # Construct probability distribution against scaled logits through softmax activation function
        probs = torch.softmax(scaled_logits, dim=-1)

        # Select top k (beam_width) probabilities and IDs from the distribution
        top_probs, top_ids = probs.topk(beam_width)

        print(f"{candidate.normalized_score}: [{candidate.tokens()}], next token probabilities:")
        for p, w in zip(top_probs.tolist()[0], tokenizer.convert_ids_to_tokens(top_ids.tolist()[0])):
          print(f"\tp: {p: .8f}: {w}")
        print("\n")

        token_counts = Counter(t.token_id for t in candidate)

        # For each of the top-k generated tokens, append to this candidate sequence,
        # update its score, and append to the list of next step candidates
        for i in range(beam_width):
          next_token_id = top_ids[:, i].item() # the new token ID
          next_score = torch.log(top_probs[:, i]).item() # log-prob of the above token

          # Optionally apply a token-specific score decay to repeated tokens
          if decay_repeated and next_token_id in token_counts:
            count = token_counts[next_token_id]
            decay = 1 + math.log(count + 1)
            print(f"{tokenizer.convert_ids_to_tokens([next_token_id])} count: {count} decay: {decay}, score: {next_score}, next: {next_score * decay}")
            next_score *= decay

          new_seq = deepcopy(candidate)

          # Adds the new token to the end of this sequence, and updates its raw and normalized scores
          # Scores are normalized by sequence token length, to avoid penalizing longer sequences
          new_seq.append(ScoredToken(next_token_id, next_score))

          # Append the updated sequence to the next candidate sequence set
          next_step_candidates.append(new_seq)
      else:
        # Append the canddiate sequence as-is to the next-step candidates if it already contains an end-of-sequence token
        next_step_candidates.append(candidate)

    print(f"next step candidates:")
    for seq in reversed(sorted(next_step_candidates)):
      print(f"\t{seq.normalized_score: .8f}: [{seq.tokens()}]")
    print("\n")

    # Sort the next-step candidates by their score, select the top-k (beam_width) scoring sequences
    # and make them the new candidate_sequences list
    next_step_candidates.sort()
    candidate_sequences = list(reversed(next_step_candidates))[:beam_width]

    # Break if all sequences in the heap end with the eos_token_id
    if all(seq.has_ended() for seq in candidate_sequences):
      break

  return candidate_sequences

In [6]:
import argparse
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer
import torch

MODEL_ID = 't5-small'
TOKENIZER_ID = 'google-t5/t5-small'

# if torch.backends.mps.is_available():
# device =  torch.device("mps")
device = "cpu"
# elif torch.backends.cuda.is_available:
#       device = torch.device("cuda")

# def detect_device(device=None):
#   """
#   Depending on the provided arguments and the available device backends, returns either a CPU,
#   or GPU device, where the GPU device may be either a CUDA-based GPU or Apple Silicon "MPS" based GPU.
#   Default device is CPU.
#   """
#   if device == "cpu":
#     return torch.device("cpu")
  
#   if not device or device == "gpu":
#     if torch.backends.mps.is_available():
#       return torch.device("mps")
#     elif torch.backends.cuda.is_available:
#       return torch.device("cuda")

#   return torch.device("cpu")  # Fallback to CPU if no GPU device is available

def run_greedy_search(tokenizer, model, input_ids, device):
  generated_ids = greedy_search(
    model, 
    input_ids, 
    max_length=64, 
    start_token_id=tokenizer.pad_token_id, 
    end_token_id=tokenizer.eos_token_id, 
    device=device
  )

  # Decode the token IDs back to strings
  decoded_text = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True)
  
  print(f"greedy search generation results:\n[\n{decoded_text}\n]")

def run_beam_search(tokenizer, model, input_ids, device, width, temperature=1.0, decay=False):
  beam_ids = beam_search(
    tokenizer, 
    model, 
    input_ids, 
    # T5 uses the pad_token_id as the starting token for 
    # text generation (https://huggingface.co/docs/transformers/main/en/model_doc/t5)
    start_token_id=tokenizer.pad_token_id,
    end_token_id=tokenizer.eos_token_id, 
    device=device, 
    beam_width=width, 
    temperature=temperature,
    decay_repeated=decay
  )

  # Select the best-scoring sequence out of the candidate set
  best_ids = beam_ids[0].ids()

  # Decode the token IDs back to strings
  decoded_text = tokenizer.decode(torch.tensor(best_ids), skip_special_tokens=True)

  print(f"beam search (k={width}, t={temperature}) generation results: [{decoded_text}]")

# def main():
#   parser = argparse.ArgumentParser()
#   parser.add_argument('--beam', type=int, required=False, help='Specify beam search width [default: 8]')
#   parser.add_argument('--decay', action='store_true', help='Apply a score decay to repeated tokens during generation')
#   parser.add_argument('--input', type=str, required=True, help='Input text file')
#   parser.add_argument('--greedy', action='store_true', help='Run greedy search for text generation')
#   parser.add_argument('--max_length', type=int, required=False, help='Maximum token length of generated output [default: 64]')
#   parser.add_argument('--prompt', type=str, required=True, help='Prompt to prefix input text with')
#   parser.add_argument('--temperature', type=float, required=False, help='Temperature setting [default: 1.0]')
#   args = parser.parse_args()

#   # device = detect_device("gpu")

In [7]:




input = '''"The quick brown fox jumps over the lazy dog" is an English-language pangram – 
a sentence that contains all the letters of the alphabet. The phrase is commonly used for 
touch-typing practice, testing typewriters and computer keyboards, displaying examples of fonts, 
and other applications involving text where the use of all letters in the alphabet is desired.
The earliest known appearance of the phrase was in The Boston Journal. 
In an article titled "Current Notes" in the February 9, 1885, edition, the phrase is mentioned as 
a good practice sentence for writing students: 
"A favorite copy set by writing teachers for their pupils is the following, because it contains 
every letter of the alphabet: 
'A quick brown fox jumps over the lazy dog.'" Dozens of other newspapers published the phrase over 
the next few months, all using the version of the sentence starting with "A" rather than "The". 
The earliest known use of the phrase starting with "The" is from the 1888 book Illustrative Shorthand 
by Linda Bronson. The modern form (starting with "The") became more common even though it is 
slightly longer than the original (starting with "A"). A 1908 edition of the Los Angeles Herald Sunday 
Magazine records that when the New York Herald was equipping an office with typewriters "a few years ago", 
staff found that the common practice sentence of "now is the time for all good men to come to the aid of
the party" did not familiarize typists with the entire alphabet, and ran onto two lines in a newspaper 
column. They write that a staff member named Arthur F. Curtis invented the "quick brown fox" pangram to 
address this. As the use of typewriters grew in the late 19th century, the phrase began appearing in 
typing lesson books as a practice sentence. Early examples include How to Become Expert in 
Typewriting: A Complete Instructor Designed Especially for the Remington Typewriter (1890), 
and Typewriting Instructor and Stenographer's Hand-book (1892). By the turn of the 20th century, 
the phrase had become widely known. In the January 10, 1903, issue of Pitman's Phonetic Journal, 
it is referred to as "the well known memorized typing line embracing all the letters of the alphabet". 
Robert Baden-Powell's book Scouting for Boys (1908) uses the phrase as a practice sentence for signaling. 
The first message sent on the Moscow–Washington hotline on August 30, 1963, was the test 
phrase "THE QUICK BROWN FOX JUMPED OVER THE LAZY DOG'S BACK 1234567890". Later, during testing, the 
Russian translators sent a message asking their American counterparts, "What does it mean when your 
people say 'The quick brown fox jumped over the lazy dog'?" During the 20th century, technicians tested 
typewriters and teleprinters by typing the sentence. It is the sentence used in the annual Zaner-Bloser 
National Handwriting Competition, a cursive writing competition which has been held in the U S. since 1991. 
In the age of computers, this pangram is commonly used to display font samples and for testing computer 
keyboards. In cryptography, it is commonly used as a test vector for hash and encryption algorithms to 
verify their implementation, as well as to ensure alphabetic character set compatibility. Microsoft Word 
has a command to auto-type the sentence, in versions up to Word 2003, using the command =rand(), and in 
Microsoft Office Word 2007 and later using the command =rand old(). Numerous references to the phrase have 
occurred in movies, television, books, video games, advertising, websites, and graphic arts. The 
lipogrammatic novel Ella Minnow Pea by Mark Dunn is built entirely around the "quick brown fox" pangram and 
its inventor. It depicts a fictional island off the South Carolina coast that idealizes the pangram, 
chronicling the effects on literature and social structure as various letters are banned from daily use by 
government dictum.With 35 letters, this is not the shortest pangram. Shorter examples include: "Waltz, bad 
nymph, for quick jigs vex." (28 letters) "Sphinx of black quartz, judge my vow." (29 letters) "How vexingly
quick daft zebras jump!" (30 letters) "Pack my box with five dozen liquor jugs." (32 letters) If 
abbreviations and non-dictionary words are allowed, it is possible to create a perfect pangram that uses 
each letter only once, such as "Mr. Jock, TV quiz PhD, bags few lynx"."'''


temperature = 1.0
decay = True
max_length = 64
prompt = "Summarize the following content: "

tokenizer = T5Tokenizer.from_pretrained(TOKENIZER_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

# model.to(device)
model.eval()

# text = open(args.input).read()

# if args.prompt:
# Prepend a prompt in front of the input text
text = f"{prompt}:\n" + input

# Encode the text into a sequence of IDs
input_ids = tokenizer.encode(text, return_tensors="pt") #.to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

2025-09-01 01:17:40.491492: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756689460.727352      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756689460.800005      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1055 > 512). Running this sequence through the model will result in indexing errors


In [8]:

run_greedy_search(tokenizer, model, input_ids, device)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


greedy search generation results:
[
the phrase, which is used in the annual Zaner-Bloser National Handwriting Competition. The phrase is used in typing lessons books, typing font samples, and other applications involving text. the phrase is commonly used for touch-typing practice, testing typewriters and keyboards
]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

def plot_logits_and_multiple_probabilities(logits, t_list, probabilities_list):
  """Plot logits and multiple softmax probability distributions on the same graph."""
  # Set up the figure and axis
  fig, ax1 = plt.subplots(figsize=(10, 5))

  # Create bar chart for logits
  indices = np.arange(len(logits))  # the x locations for the groups
  ax1.bar(indices, logits, width=0.35, label='Logit values', color='#89CFF0', alpha=0.7)
  ax1.set_xlabel('Class index')
  ax1.set_ylabel('Logit values', color='#89CFF0')
  ax1.tick_params('y', colors='#89CFF0')

  # Create a second y-axis for the softmax probabilities
  ax2 = ax1.twinx()
  
  colors = ['g', 'r', 'b', 'y', 'm', 'c', 'k']  # Define a list of colors for the lines
  for i, probabilities in enumerate(probabilities_list):
    if t_list[i] == 1.0:
      ax2.plot(indices, probabilities, linewidth=4.0, label=f'Scaled Post-Softmax Probability T={t_list[i]}', color=colors[i % len(colors)])
    else:    
      ax2.plot(indices, probabilities, label=f'Scaled Post-Softmax Probability T={t_list[i]}', color=colors[i % len(colors)])
  
  ax2.set_ylabel('Class Probability', color='r')
  ax2.tick_params('y', colors='r')

  # Title and legend
  plt.title('Model Logits vs Temperature-Scaled Probabilities')
  fig.tight_layout()
  ax1.legend(loc='upper left')
  ax2.legend(loc='upper right')

  # Show the plot
  plt.show()

# Example logits
ts = [0.5, 1.0, 2.0, 4.0, 8.0]
logits = torch.tensor([3.1230, 5.0000, 3.2340, 2.6420, 2.4660, 3.3532, 3.8000, 2.9110])
probs = [torch.softmax(logits / t, dim=-1) for t in ts]

# Plotting the graph
plot_logits_and_multiple_probabilities(logits, ts, probs)

In [9]:
beam = 8
run_beam_search(tokenizer, model, input_ids, device, beam, temperature , decay)

beam search | k = 8 bos = 0 eos = 1 temp = 1.0 beam_width = 8


  0%|          | 0/64 [00:00<?, ?it/s]

0.0: [], next token probabilities:
	p:  0.99999630: <extra_id_0>
	p:  0.00000054: ▁
	p:  0.00000046: ▁"
	p:  0.00000024: <extra_id_3>
	p:  0.00000020: <extra_id_19>
	p:  0.00000018: <extra_id_1>
	p:  0.00000013: <extra_id_2>
	p:  0.00000011: ▁The




  2%|▏         | 1/64 [00:01<01:17,  1.23s/it]

next step candidates:
	-0.00000185: []
	-7.21584177: []
	-7.29066229: ["]
	-7.61841726: []
	-7.70325518: []
	-7.75997686: []
	-7.92314005: []
	-7.99026155: [The]


-1.847747398642241e-06: [], next token probabilities:
	p:  0.10696564: ▁the
	p:  0.09694867: ,
	p:  0.07409322: :
	p:  0.05762960: ▁is
	p:  0.04704630: ▁"
	p:  0.04456405: ▁
	p:  0.04071314: ▁in
	p:  0.03783220: ▁of


-7.215841770172119: [], next token probabilities:
	p:  0.45392263: a
	p:  0.39019713: earliest
	p:  0.04012274: '
	p:  0.00871329: .
	p:  0.00441259: s
	p:  0.00407035: he
	p:  0.00273233: <unk>
	p:  0.00240624: th


-7.2906622886657715: ["], next token probabilities:
	p:  0.55122370: The
	p:  0.18449318: the
	p:  0.06050230: qui
	p:  0.02253170: A
	p:  0.01472596: a
	p:  0.00527774: Qu
	p:  0.00524824: ▁The
	p:  0.00515081: C


-7.618417263031006: [], next token probabilities:
	p:  0.14472343: ▁is
	p:  0.11007494: :
	p:  0.06776276: ▁"
	p:  0.06406648: ,
	p:  0.05292276: ▁of
	p:  0.03802692: ▁
	p:  0.02980479:

...

	-0.35105114: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice, testing typewriters and keyboards. it is also commonly]
	-0.35559076: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice, testing typewriters and computer keyboards. it was used]
	-0.36024166: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice, testing typewriters and keyboards. it is also known]
	-0.36337744: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice

 98%|█████████▊| 63/64 [11:49<00:06,  6.62s/it]

-0.28662687365270645: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice, testing typewriters and computer keyboards. it is commonly used], next token probabilities:
	p:  0.39177215: ▁to
	p:  0.39126837: ▁for
	p:  0.13989307: ▁in
	p:  0.04001318: ▁as
	p:  0.00653563: ▁by
	p:  0.00492660: ▁on
	p:  0.00395595: ▁during
	p:  0.00218803: ▁when


['▁for'] count: 1 decay: 1.6931471805599454, score: -0.9383615851402283, next: -1.5887842722259387
-0.291387037095413: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice, testing typewriters and computer keyboards. it is also used], next token probabilities:
	p:  0.36734861: ▁to
	p:  0.27997202: ▁for
	p:  0.15800259: ▁as
	p:  0.14904018: ▁in
	p:  0.01292066: ▁by
	p:  0

100%|██████████| 64/64 [11:54<00:00, 11.16s/it]

	-0.46955328: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice, testing typewriters and keyboards. it is also used to]


beam search (k=8, t=1.0) generation results: [: "The quick brown fox jumps over the lazy dog" is an English-language pangram – a sentence that contains all the letters of the alphabet. The phrase is commonly used for touch-typing practice, testing typewriters and computer keyboards.]
